# Tech Layoffs Data Cleaning in SQL

## Project Goal
This project analyzes global tech layoffs from 2020–2024, uncovering trends across companies, industries, and countries to provide actionable insights for business leaders, investors, and policy makers.

I used **SQL (SQLite3)** to clean this **[Tech layoffs 2020 - 2024](https://www.kaggle.com/datasets/ulrikeherold/tech-layoffs-2020-2024)** dataset, and built an **interactive Tableau dashboard** to uncover trends across companies, industries, and countries. 

For those who want to explore the work on GitHub: 

👉 [Tech Layoffs 2020–2024 on GitHub](https://github.com/Wilfrida-Were/Tech-Layoffs-Data-Cleaning-in-SQL/blob/main/README.md)

## 📑 Table of Contents
* [Interactive Dashboard](#interactive-dashboard)
* [Code](#code)
* [Key Insights](#key-insights)
* [Who Can Use This Project](#who-can-use-this-project)
* [Quick Recommendations](#quick-recommendations)
* [Tech Stack](#tech-stack)
* [Workflow](#workflow)
* [Key Learnings](#key-learnings)
* [Let’s Connect](#lets-connect)


<a id="interactive-dashboard"></a>
## 🚀 Interactive Dashboard
👉 [View the Interactive Dashboard on Tableau Public](https://public.tableau.com/app/profile/wilfrida.were/viz/TechLayoffs2020-2024_17198461868990/TECHLAYOFFS2020-2024)  

Preview (click image to open dashboard):  
[![Tech Layoffs Dashboard](https://raw.githubusercontent.com/Wilfrida-Were/Tech-Layoffs-Data-Cleaning-in-SQL/7204e10b6aff469807b4e416e4237384e33482f1/TECH%20LAYOFFS%202020-2024.png)](https://public.tableau.com/app/profile/wilfrida.were/viz/TechLayoffs2020-2024_17198461868990/TECHLAYOFFS2020-2024)  

<a id="code"></a>
## 💻 Code

In [1]:
import numpy as np
import pandas as pd
import sqlite3 # for using SQL commands

In [2]:
# Load the data
tech_layoffs = pd.read_excel('/kaggle/input/tech-layoffs-2020-2024/tech_layoffs.xlsx')
df = tech_layoffs.copy() # make a copy
df.head()

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18,10.0,1000.0,900.0,Retail,Unknown,NaN,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.77493,-122.41942


In [3]:
df.info()  # 1672 rows and 16 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672 entries, 0 to 1671
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   #                            1672 non-null   int64         
 1   Company                      1672 non-null   object        
 2   Location_HQ                  1672 non-null   object        
 3   Country                      1672 non-null   object        
 4   Continent                    1672 non-null   object        
 5   Laid_Off                     1565 non-null   float64       
 6   Date_layoffs                 1672 non-null   datetime64[ns]
 7   Percentage                   1570 non-null   float64       
 8   Company_Size_before_Layoffs  1511 non-null   float64       
 9   Company_Size_after_layoffs   1536 non-null   float64       
 10  Industry                     1672 non-null   object        
 11  Stage                        1672 non-null 

The columns `Laid_Off`, `Percentage`, `Company_Size_before_Layoffs`, `Company_Size_after_layoffs` have missing values so I will inspect them. This helps decide whether to impute or drop those rows with missing values. I don't need the `Money_Raised_in_$_mil` column

## Connect to a Database and convert the Dataframe to a Table

In [4]:
# Connect to the database
conn = sqlite3.connect('tech_layoffs_data.db')

In [5]:
# Create 'tech_layoffs' table
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='tech_layoffs'")
table_exists = len(cursor.fetchall()) > 0

if not table_exists:
  # Load data to 'tech_layoffs' table only if it doesn't exist
  df.to_sql('tech_layoffs', conn, index=False)
  print("Table 'tech_layoffs' created and data loaded successfully.")
else:
  print("Table 'tech_layoffs' already exists. Skipping data loading.")

Table 'tech_layoffs' created and data loaded successfully.


In [6]:
# Now I will use SQL commands to Clean data from the 'tech_layoffs' table
# pd.read_sql_query is an optimized approach for working with pandas and sqlite3

# Inspect the first_10_rows
first_10_rows = pd.read_sql_query("""SELECT *
                                   FROM tech_layoffs
                                   LIMIT 10;""", conn)
first_10_rows

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20 00:00:00,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19 00:00:00,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18 00:00:00,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18 00:00:00,10.0,1000.0,900.0,Retail,Unknown,None,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14 00:00:00,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.77493,-122.41942
5,16,Bolt,San Francisco Bay Area,USA,North America,130.0,2023-12-14 00:00:00,29.0,450.0,320.0,Finance,Series E,$1300,2023,37.77493,-122.41942
6,20,Invitae,San Francisco Bay Area,USA,North America,235.0,2023-12-13 00:00:00,15.0,1567.0,1332.0,Healthcare,Post-IPO,None,2023,37.77493,-122.41942
7,21,Etsy,New York City,USA,North America,225.0,2023-12-13 00:00:00,11.0,2045.0,1820.0,Retail,Post-IPO,$97,2023,40.71427,-74.00597
8,27,Chipper Cash,San Francisco Bay Area,USA,North America,15.0,2023-12-11 00:00:00,33.0,45.0,30.0,Finance,Series C,$302,2023,37.77493,-122.41942
9,31,Zulily,Seattle,USA,North America,839.0,2023-12-08 00:00:00,100.0,839.0,0.0,Retail,Acquired,$194,2023,47.60621,-122.33207


## Data Cleaning

## Identify Spelling Errors

In [7]:
# Inspect the Industry column 
industry = pd.read_sql_query("""
SELECT DISTINCT Industry
FROM tech_layoffs;
""",conn)

industry

,Industry
0,Consumer
1,Healthcare
2,Energy
3,Retail
4,Transportation
5,Finance
6,Product
7,Media
8,Other
9,AI


## Update spelling mistakes in the Industry column

These industries will be **updated**:

* Transport… &#8594; Transportation
* Manufact… &#8594; Manufacturing
* Infrastruc… &#8594; Infrastructure
* Construct… &#8594; Construction

In [8]:
# Update to Transportation
update_query = """
UPDATE tech_layoffs
SET Industry = 'Transportation'
WHERE Industry LIKE 'Transport%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

In [9]:
# Update to Manufacturing
update_query = """
UPDATE tech_layoffs
SET Industry = 'Manufacturing'
WHERE Industry LIKE 'Manufact%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

In [10]:
# Update to Infrastructure
update_query = """
UPDATE tech_layoffs
SET Industry = 'Infrastructure'
WHERE Industry LIKE 'Infrastruc%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

In [11]:
# Update to Construction
update_query = """
UPDATE tech_layoffs
SET Industry = 'Construction'
WHERE Industry LIKE 'Construct%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

In [12]:
# Confirm the Updates
industry_updates = pd.read_sql_query("""
SELECT DISTINCT Industry 
FROM tech_layoffs
""",conn)

industry_updates

,Industry
0,Consumer
1,Healthcare
2,Energy
3,Retail
4,Transportation
5,Finance
6,Product
7,Media
8,Other
9,AI


## Duplicate Rows

In [13]:
# Identify rows that have duplicate values in all these columns. I will drop the duplicate rows later
# Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry

duplicate_rows = pd.read_sql_query("""
SELECT *
FROM (
  SELECT Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry,
         ROW_NUMBER() OVER (PARTITION BY Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry) AS row_num
  FROM tech_layoffs
) AS duplicates  
WHERE row_num > 1;
""", conn)

duplicate_rows

,Company,Location_HQ,Country,Laid_Off,Date_layoffs,Industry,row_num
0,Beyond Meat,Los Angeles,USA,200.0,2022-10-14 00:00:00,Food,2
1,Cazoo,London,United Kingdom,750.0,2022-06-07 00:00:00,Transportation,2
2,Chipper Cash,San Francisco Bay Area,USA,15.0,2023-12-11 00:00:00,Finance,2
3,Cruise,San Francisco Bay Area,USA,900.0,2023-12-14 00:00:00,Transportation,2
4,Enphase Energy,San Francisco Bay Area,USA,350.0,2023-12-18 00:00:00,Energy,2
5,Etsy,New York City,USA,225.0,2023-12-13 00:00:00,Retail,2
6,Invitae,San Francisco Bay Area,USA,235.0,2023-12-13 00:00:00,Healthcare,2
7,Oda,Oslo,Norway,70.0,2022-11-01 00:00:00,Food,2
8,ShareChat,Bengaluru,India,200.0,2023-12-20 00:00:00,Consumer,2
9,Udaan,Bengaluru,India,100.0,2023-12-18 00:00:00,Retail,2


In [14]:
# Inspect any of the duplicate rows
Beyond_meat = pd.read_sql_query("""
SELECT Company, Location_HQ, Country, Laid_Off,Date_layoffs, Industry
FROM tech_layoffs
WHERE Company LIKE 'Beyond Meat';
""",conn)

Beyond_meat

,Company,Location_HQ,Country,Laid_Off,Date_layoffs,Industry
0,Beyond Meat,Los Angeles,USA,65.0,2023-11-02 00:00:00,Food
1,Beyond Meat,Los Angeles,USA,200.0,2022-10-14 00:00:00,Food
2,Beyond Meat,Los Angeles,USA,200.0,2022-10-14 00:00:00,Food
3,Beyond Meat,Los Angeles,USA,40.0,2022-08-03 00:00:00,Food


In [15]:
# Inspect another duplicate row
Chipper_cash = pd.read_sql_query("""
SELECT Company, Location_HQ, Country, Laid_Off,Date_layoffs, Industry
FROM tech_layoffs
WHERE Company LIKE 'Chipper Cash'
""",conn)

Chipper_cash

,Company,Location_HQ,Country,Laid_Off,Date_layoffs,Industry
0,Chipper Cash,San Francisco Bay Area,USA,15.0,2023-12-11 00:00:00,Finance
1,Chipper Cash,San Francisco Bay Area,USA,100.0,2023-02-17 00:00:00,Finance
2,Chipper Cash,San Francisco Bay Area,USA,50.0,2022-12-06 00:00:00,Finance
3,Chipper Cash,San Francisco Bay Area,USA,20.0,2024-03-15 00:00:00,Finance
4,Chipper Cash,San Francisco Bay Area,USA,15.0,2023-12-11 00:00:00,Finance


## Missing Values

In [16]:
# Missing values in Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs columns
missing_values = pd.read_sql_query("""
SELECT Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs
FROM tech_layoffs
WHERE Laid_Off IS NULL
OR Percentage IS NULL 
OR Company_Size_before_Layoffs IS NULL 
OR Company_Size_after_layoffs IS NULL;
""",conn)

for col in missing_values.columns:
    print(f"Missing values in '{col}': {missing_values[col].isnull().sum()}")

print() # blank line
missing_values.head(10) # the first 10 rows

Missing values in 'Laid_Off': 107
Missing values in 'Percentage': 102
Missing values in 'Company_Size_before_Layoffs': 161
Missing values in 'Company_Size_after_layoffs': 136



,Laid_Off,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs
0,NaN,NaN,None,NaN
1,NaN,NaN,None,NaN
2,100.0,NaN,None,0.0
3,NaN,20.0,None,NaN
4,NaN,5.0,None,NaN
5,54.0,NaN,None,NaN
6,20.0,NaN,None,NaN
7,NaN,100.0,None,0.0
8,NaN,NaN,None,NaN
9,NaN,40.0,None,NaN


## The Problem with Imputation

In this case, the columns `Laid_Off`, `Percentage`, `Company_Size_before_Layoffs`, and `Company_Size_after_layoffs` are all **related**.

Imagine these rows represent companies that went through layoffs.  Since we don't have all the information *(number laid off, company size before/after)*,  trying to guess those missing values would be like making up a story.  It's better to remove these rows and focus on the companies where we have complete data for a more accurate analysis.

In [17]:
# FILTER the data, EXCLUDING rows with missing values in the specified columns
Cleaned_tech_layoffs = pd.read_sql_query("""
SELECT *
FROM tech_layoffs
WHERE Laid_Off IS NOT NULL
AND Percentage IS NOT NULL 
AND Company_Size_before_Layoffs IS NOT NULL 
AND Company_Size_after_layoffs IS NOT NULL;
""",conn)

print(Cleaned_tech_layoffs.shape) # 1511 clean rows remain after Filtering 
print() # blank line
Cleaned_tech_layoffs.head()

(1511, 16)



,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20 00:00:00,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19 00:00:00,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18 00:00:00,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18 00:00:00,10.0,1000.0,900.0,Retail,Unknown,None,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14 00:00:00,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.77493,-122.41942


In [18]:
# Drop DUPLICATE rows from the Dataframe, based on the columns used earlier when inspecting Duplicates
Cleaned_tech_layoffs = Cleaned_tech_layoffs.drop_duplicates(subset=['Company', 'Location_HQ', 'Country', 'Laid_Off',
                                                                    'Date_layoffs','Industry'], keep='first') # keep first instances

print(Cleaned_tech_layoffs.shape) # 1501 rows remain after dropping duplicate rows

(1501, 16)


In [19]:
# Save the cleaned DataFrame to a CSV file
Cleaned_tech_layoffs.to_csv('Cleaned_tech_layoffs.csv', index=False)
print('Cleaned_tech_layoffs.csv saved.')

Cleaned_tech_layoffs.csv saved.


In [20]:
# Close the connection
conn.commit()  # Save changes if any
conn.close()

<a id="key-insights"></a>
## 🔑 Key Insights

- **📉 Total Layoffs:** 409,323 employees affected worldwide.  
- **🌍 By Country:**  
  - USA dominates with ~ 305K layoffs (~74%).  
  - Other impacted countries: India (27K), Germany (23K), and the UK (13K).  
- **📈 Trend Over Time:**  
  - Lowest layoffs: 2021 (~6.7K).  
  - Peak: 2023 (174K).  
  - Sharp decline in 2024 (~49K).  
- **🏢 By Company:** Amazon (27K), Meta (21K), and Google (12K) top the list, followed by SAP and Microsoft (10K each).  
- **🏭 By Industry:** Retail (56K) and Consumer tech (55K) are most affected, followed by Transportation (34K) and Food tech (30K).  
- **⚠️ Extreme Cases:** Startups like Zulily, ZestMoney, Udayy, and TutorMundi laid off **100% of their staff** in specific years.  


<a id="who-can-use-this-project"></a>
## 👥 Who Can Use This Project?

- **Job Seekers** → Identify industries and regions that have been more stable and target opportunities accordingly.  
- **Investors & Analysts** → Understand sector-level risks and spot trends to inform investment decisions.
- **Founders / Business Leaders** → Benchmark company or industry performance to inform hiring or scaling strategies. 
- **Policy Makers** → Design reskilling and support programs for regions or industries disproportionately affected. 
- **Educators & Learners** → Use the dataset, SQL scripts, and dashboard as a case study for data analysis, visualization, and business insights.

<a id="quick-recommendations"></a>
## ✅ Quick Recommendations

- **For Job Seekers:** Focus on industries that have shown relative stability, such as **Finance and Hardware**. Understanding which sectors are less affected by layoffs can help you target applications more strategically and avoid volatile sectors 
- **For Investors:** Be cautious with **Consumer and Retail** tech sectors, which have seen higher layoffs. Consider diversifying your investments across more stable industries to reduce exposure to market volatility.  
- **For Founders:** Use industry-wide layoff trends to **inform hiring and scaling decisions**. In sectors with high layoffs, consider conservative hiring strategies, flexible staffing, or scenario planning to mitigate business risk.  
- **For Policy Makers:** Target **reskilling and workforce support programs** in regions most affected by layoffs. Those regions could benefit from training programs in emerging or stable sectors to improve employment outcomes. 

<a id="tech-stack"></a>
## ⚙️ Tech Stack
- **SQL (SQLite3)** – Data cleaning and preprocessing  
- **Tableau** – Interactive dashboards and visualization  
- **Kaggle Notebook** – End-to-end workflow with code and explanations  

<a id="workflow"></a>
## 🔄 Workflow
1. **Raw Data** → [Kaggle Dataset: Tech Layoffs 2020–2024](https://www.kaggle.com/datasets/ulrikeherold/tech-layoffs-2020-2024)  
2. **Data Cleaning** → Performed with SQL (SQLite3).  
3. **Exploration** → SQL queries for aggregation & trend analysis.  
4. **Visualization** → Built Tableau dashboard for insights and storytelling. 

<a id="key-learnings"></a>
## 📌 Key Learnings
- Handling missing and inconsistent data in SQL.  
- Structuring cleaned datasets for visualization.  
- Designing dashboards that highlight **business-relevant insights**. 

<a id="lets-connect"></a>
## 🔗 Let’s Connect

If my work was helpful or inspiring, feel free to connect, follow, or support:  

[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect-blue?style=flat&logo=linkedin)](https://linkedin.com/in/wilfridawere/)  
[![Website](https://img.shields.io/badge/Website-Visit-orange?style=flat&logo=google-chrome)](https://www.wilfridawere.com/)  
[![Kaggle](https://img.shields.io/badge/Kaggle-Follow-blue?style=flat&logo=kaggle)](https://kaggle.com/wilfridawere)  
[![GitHub](https://img.shields.io/badge/GitHub-Projects-black?style=flat&logo=github)](https://github.com/Wilfrida-Were)  
[![Ko-fi](https://img.shields.io/badge/Ko--fi-Support-red?style=flat&logo=ko-fi)](https://ko-fi.com/wilfridawere/)  